In [7]:
import numpy as np
import pandas as pd

In [8]:
import plotly.graph_objs as go


In [55]:
def gen_bar_plotly(keys, df):
    if isinstance(df, dict):
        df = pd.DataFrame(df)
    names = df.columns
    fig = go.Figure(data=[go.Bar(x=keys, y=df[name], name=name) for name in names])
    fig.show()
def gen_bar_plotly_by_counter(c):
    _tdf = pd.DataFrame({'k': [x for x in c], 'v': [c.get(x) for x in c]})
    _tdf1 = _tdf.sort_values(by='v')
    fig = go.Figure(data=[go.Bar(x=_tdf1.k, y=_tdf1.v)])
    fig.show()

In [10]:
df = pd.read_csv('../data/samecar.201908.csv')

In [11]:
df['is_same'] = df.is_same.map(lambda x: x if x == 1 else 0)

In [12]:
len(df),  sum(df.is_same), set(df.is_same)

(10718, 3553, {0, 1})

In [13]:
df.head()

,id,color1,color2,displacement1,displacement2,mile1,mile2,year1,year2,licensed_city1,...,owner_price2,is_import1,is_import2,transfer_times1,transfer_times2,displacement_standard1,displacement_standard2,gearbox1,gearbox2,is_same
0,1,白色,白色,1.4,NaN,4.75,9.1,2011.0,NaN,北京,...,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,自动,0
1,2,灰色,黑色,2.0,NaN,5.22,7.9,2012.0,NaN,上海,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,自动,0
2,3,灰色,其他色,1.6,NaN,6.86,15.7,2011.0,NaN,哈尔滨,...,NaN,0.0,0.0,0.0,2.0,NaN,NaN,NaN,自动,0
3,4,白色,其他色,2.0,NaN,4.13,15.3,2012.0,NaN,南京,...,NaN,0.0,0.0,0.0,1.0,NaN,NaN,NaN,自动,0
4,5,黑色,黑色,1.8,NaN,11.16,7.3,2013.0,NaN,北京,...,NaN,0.0,0.0,0.0,0.0,NaN,NaN,NaN,自动,0


In [15]:
from collections import Counter

In [33]:
color1 = Counter(df.color1)
color2 = Counter(df.color2)
colors = list(set(df.color1).union( set(df.color2)))
colord = {
    'color1': [color1.get(color, 0) for color in colors],
    'color2': [color2.get(color, 0) for color in colors],
}

In [34]:
gen_bar_plotly(colors, colord)

In [ ]:
# 颜色本身就属于类别特征
# 直接把color1, color2 输入就可以树模型或者贝叶斯模型就可以
# 线性模型还需要特别处理

In [35]:
df.columns

Index(['id', 'color1', 'color2', 'displacement1', 'displacement2', 'mile1',
       'mile2', 'year1', 'year2', 'licensed_city1', 'licensed_city2', 'title1',
       'title2', 'fuel_type1', 'fuel_type2', 'register_time1',
       'register_time2', 'owner_price1', 'owner_price2', 'is_import1',
       'is_import2', 'transfer_times1', 'transfer_times2',
       'displacement_standard1', 'displacement_standard2', 'gearbox1',
       'gearbox2', 'is_same'],
      dtype='object')

In [36]:
rdf = df.copy()

In [37]:
rdf['displacement_diff'] = rdf.displacement1 - rdf.displacement2

In [40]:
a = rdf.displacement_diff[rdf.is_same == 0].map(lambda x: round(x, 2))
b = rdf.displacement_diff[rdf.is_same == 1].map(lambda x: round(x, 2))
# 10 代表空
ac = Counter(list(map(lambda x: x if x == x else 10, a)))
bc = Counter(list(map(lambda x: x if x == x else 10, b)))

In [65]:
gen_bar_plotly_by_counter(ac)

In [66]:
gen_bar_plotly_by_counter(bc)

In [87]:
rdf['displacement_diff_sparse'] = rdf.displacement_diff.map(lambda x: (2 if x ==0 else 0) if x == x else 0.5)

In [88]:
# 离散化后的displacement
a = rdf.displacement_diff_sparse[rdf.is_same == 0].map(lambda x: round(x, 2))
b = rdf.displacement_diff_sparse[rdf.is_same == 1].map(lambda x: round(x, 2))
# 10 代表空
ac = Counter(a)
bc = Counter(b)
# 非共有车的分布
gen_bar_plotly_by_counter(ac)

In [89]:
# 共有车的分布
gen_bar_plotly_by_counter(bc)

In [74]:
rdf['mile_diff_rate'] = (rdf.mile1 - rdf.mile2) / (rdf.mile1 + rdf.mile2)
a = rdf[rdf.is_same == 0].mile_diff_rate.map(lambda x: round(x, 2))
b = rdf[rdf.is_same == 1].mile_diff_rate.map(lambda x: round(x, 2))
ac = Counter(a)
bc = Counter(b)

In [75]:
# 非共有车里程差距比例的分布
gen_bar_plotly_by_counter(ac)

In [73]:
# 共有车里程差距比例的分布
gen_bar_plotly_by_counter(bc)

In [90]:
def mile_diff_rate_sparse(x):
    if x == x:
        if x == 0:
            return 2
        if abs(x) < 0.1:
            return 1
        return 0
    return 0.5
rdf['mile_diff_rate_sparse'] = rdf.mile_diff_rate.map(mile_diff_rate_sparse)

In [85]:
rdf['mile_diff'] = (rdf.mile1 - rdf.mile2)
a = rdf[rdf.is_same == 0].mile_diff.map(lambda x: int(x) if x == x else 100)
b = rdf[rdf.is_same == 1].mile_diff.map(lambda x: int(x) if x == x else 100)
ac = Counter(a)
bc = Counter(b)

In [82]:
gen_bar_plotly_by_counter(ac)

In [86]:
gen_bar_plotly_by_counter(bc)

In [91]:
def mile_diff_sparse(x):
    if x == x:
        if x == 0:
            return 2
        if abs(x) <= 2:
            return 1
        return 0
    return 0.5
rdf['mile_diff_sparse'] = rdf.mile_diff.map(mile_diff_sparse)

In [92]:
rdf['year_diff'] = (rdf.year1 - rdf.year2)
a = rdf[rdf.is_same == 0].year_diff.map(lambda x: int(x) if x == x else 100)
b = rdf[rdf.is_same == 1].year_diff.map(lambda x: int(x) if x == x else 100)
ac = Counter(a)
bc = Counter(b)

In [93]:
gen_bar_plotly_by_counter(ac)

In [94]:
gen_bar_plotly_by_counter(bc)

In [95]:
def year_diff_sparse(x):
    if x == x:
        if x==0:
            return 2
        return 0
    return 0.5
rdf['year_diff_sparse'] = rdf.year_diff.map(year_diff_sparse)

In [96]:
set(rdf.licensed_city1), set(rdf.licensed_city2)

({'R重庆',
  nan,
  '万州区',
  '上海',
  '上海#',
  '上饶',
  '东莞',
  '东莞店',
  '东营',
  '中山',
  '临汾',
  '临沂',
  '丹东',
  '丽水',
  '乌兰察布',
  '乌海',
  '乌鲁木齐',
  '乐山',
  '九江',
  '云浮',
  '亳州',
  '仙桃',
  '佛山',
  '佛山南海',
  '佛山南海区',
  '佛山顺德',
  '佳木斯',
  '保定',
  '信阳',
  '六安',
  '六盘水',
  '兰州',
  '内江',
  '包头',
  '北京',
  '北海',
  '十堰',
  '南京',
  '南充',
  '南宁',
  '南平',
  '南昌',
  '南昌/省直系统',
  '南通',
  '南阳',
  '厦门',
  '台州',
  '合肥',
  '吉安',
  '吉林',
  '周口',
  '呼和浩特',
  '呼市',
  '咸宁',
  '咸阳',
  '哈尔滨',
  '唐山',
  '商丘',
  '嘉兴',
  '四平',
  '大同',
  '大庆',
  '大连',
  '天水',
  '天津',
  '天门',
  '太原',
  '威海',
  '娄底',
  '孝感',
  '宁德',
  '宁波',
  '安庆',
  '安康',
  '安阳',
  '安顺',
  '宜宾',
  '宜昌',
  '宜春',
  '宝鸡',
  '宣城',
  '宿州',
  '宿迁',
  '岳阳',
  '崇左',
  '巢湖',
  '巴中',
  '巴彦淖尔',
  '常州',
  '常德',
  '平潭/省直系统',
  '平顶山',
  '广安',
  '广州',
  '廊坊',
  '延安',
  '开封',
  '张家口',
  '徐州',
  '德宏',
  '德州',
  '德惠',
  '德阳',
  '怀化',
  '恩施',
  '惠州',
  '成都',
  '扬州',
  '承德',
  '抚州',
  '抚顺',
  '揭阳',
  '新乡',
  '新余',
  '无锡',
  '日照',
  '昆明',
  '昌吉',
  '晋中',
  '景德镇',
  '曲靖'

In [99]:
city_clean_map = {'R重庆': '重庆', '上海#': '上海', '东莞店': '东莞',
  '南昌/省直系统': '南昌','济南/省直系统': '济南/省直系统',
  '平潭/省直系统':'平潭', '伊犁哈萨克自治州': '伊犁', '德宏傣族景颇族自治州': '德宏',
  '佛山南海': '佛山', '凉山彝族自治州': '凉山','南昌省直系统':'南昌',
  '佛山南海区': '佛山', '昌吉回族自治州': '昌吉', '杨凌高新农业示范区': '杨凌',
  '佛山顺德': '佛山', '潍坊增补': '潍坊', '红河哈尼族彝族自治州': '红河',
  '佛山顺德区': '佛山', '锡林郭勒盟': '锡林郭勒',
                 }

In [100]:
def licensed_city_diff_sparse(city1, city2):
    if city1 == city1 and city2 == city2:
        c1 = city_clean_map.get(city1, city1)
        c2 = city_clean_map.get(city2, city2)
        if c1 == c2:
            return 2
        return 0
    else:
        return 0.5
rdf['licensed_city_diff_sparse'] = rdf.apply(lambda row: licensed_city_diff_sparse(row.licensed_city1, row.licensed_city2), axis=1)

In [101]:
rdf.title2.head()

0       大众 高尔夫 2012款 2.0TSI GTI
1    福特 福克斯 2009款 两厢 1.8L 自动时尚型
2         大众 速腾 2006款 1.6L自动时尚型
3          别克 君威 2004款 2.5L 豪华型
4    大众 帕萨特 2013款 2.0TSI DSG御尊版
Name: title2, dtype: object

In [102]:
rdf.title1.head()

0     大众-高尔夫 2011款 1.4TSI 自动舒适型
1    福特-福克斯 2012款 三厢 2.0L 自动旗舰型
2        大众-速腾 2011款 1.6L 自动舒适型
3          别克-君威 2012款 2.0L 舒适版
4    大众-帕萨特 2013款 1.8TSI DSG至尊版
Name: title1, dtype: object

In [103]:
def edit_distance(word1, word2):
    """
    :type word1: str
    :type word2: str
    :rtype: int
    """
    l1 = len(word1)
    l2 = len(word2)
    if l1 == 0:
        return None
    if l2 == 0:
        return None
    l1 += 1
    l2 += 1
    mat = [[j for j in range(l2)] for i in range(l1)]
    for i in range(l1):
        mat[i][0] = i
    for i in range(1, l1):
        i_ = i - 1
        letter1 = word1[i_]
        for j in range(1, l2):
            j_ = j - 1
            letter2 = word2[j_]
            if letter1 != letter2:
                steps = mat[i_][j_] + 1
            else:
                steps = mat[i_][j_]
            if steps > mat[i][j_] + 1:
                steps = mat[i][j_] + 1
            if steps > mat[i_][j] + 1:
                steps = mat[i_][j] + 1
            mat[i][j] = steps
    return mat[l1 - 1][l2 - 1]

In [115]:
def title_diff(row):
    if row.title1 == row.title1 and row.title2 == row.title2:
        t1 = row.title1.replace(' ', '').replace('-', '')
        t2 = row.title2.replace(' ', '').replace('-', '')
        return edit_distance(t1, t2)
    return None
rdf['title_diff'] = rdf.apply(lambda row: title_diff(row), axis=1)

In [117]:
a = rdf[rdf.is_same == 0].title_diff.map(lambda x: int(x) if x == x else 100)
b = rdf[rdf.is_same == 1].title_diff.map(lambda x: int(x) if x == x else 100)
ac = Counter(a)
bc = Counter(b)

In [118]:
gen_bar_plotly_by_counter(ac)

In [119]:
gen_bar_plotly_by_counter(bc)

In [122]:
def title_diff_sparse(x):
    if x == x:
        if x == 0:
            return 2
        if x <=4:
            return 1
        return 0
    return 0.5
rdf['title_diff_sparse'] = rdf.title_diff.map(title_diff_sparse)

In [123]:
rdf['year_diff'] = rdf.year1 - rdf.year2
a = rdf[rdf.is_same == 0].year_diff.map(lambda x: int(x) if x == x else 100)
b = rdf[rdf.is_same == 1].year_diff.map(lambda x: int(x) if x == x else 100)
ac = Counter(a)
bc = Counter(b)

In [124]:
gen_bar_plotly_by_counter(ac)

In [125]:
gen_bar_plotly_by_counter(bc)

In [126]:
rdf['year_diff_sparse'] = rdf.year_diff.map(lambda x: (2 if x == 0 else 0) if x==x else 0.5)

In [127]:
df.columns

Index(['id', 'color1', 'color2', 'displacement1', 'displacement2', 'mile1',
       'mile2', 'year1', 'year2', 'licensed_city1', 'licensed_city2', 'title1',
       'title2', 'fuel_type1', 'fuel_type2', 'register_time1',
       'register_time2', 'owner_price1', 'owner_price2', 'is_import1',
       'is_import2', 'transfer_times1', 'transfer_times2',
       'displacement_standard1', 'displacement_standard2', 'gearbox1',
       'gearbox2', 'is_same'],
      dtype='object')

In [148]:
set(rdf.fuel_type1), set(rdf.fuel_type2)

({nan, '柴油', '汽油', '油改气', '油电', '电动'},
 {'1', nan, '柴油', '汽油', '油改汽', '油电', '燃油', '电动'})

In [152]:
rdf['fuel_type2'] = df.fuel_type2

In [156]:
import math
fuel_map = {
    '混动': '油电',
    '—': None,
}
rdf['fuel_type2'] = rdf.fuel_type2.map(lambda x: fuel_map.get(x, x) if x == x else None)
rdf['fuel_type1'] = rdf.fuel_type1.map(lambda x: fuel_map.get(x, x) if x == x else None)

In [157]:
set(rdf.fuel_type1), set(rdf.fuel_type2)

({None, '柴油', '汽油', '油改气', '油电', '电动'},
 {None, '柴油', '汽油', '油改汽', '油电', '燃油', '电动'})

In [160]:
def date_to_months(dt):
    if dt is not None and dt == dt:
        arr = [int(x) for x in dt.split('-')]
        return arr[0]*12 + arr[1]
    return None
date_to_months('2012-06-01')

24150

In [162]:
def register_time_diff(row):
    if row.register_time1 == row.register_time1 and row.register_time1 is not None \
    and row.register_time2 == row.register_time2 and row.register_time2 is not None:
        return date_to_months(row.register_time1) - date_to_months(row.register_time2)
    return None
rdf['register_time_diff'] = rdf.apply(register_time_diff, axis=1)

In [168]:
bar_to_show('register_time_diff')

In [167]:
def bar_to_show(key, default=100):
    a = rdf[rdf.is_same == 0][key].map(lambda x: int(x) if x == x else default)
    b = rdf[rdf.is_same == 1][key].map(lambda x: int(x) if x == x else default)
    ac = Counter(a)
    bc = Counter(b)
    gen_bar_plotly_by_counter(ac), gen_bar_plotly_by_counter(bc)

In [166]:
def register_time_diff_sparse(x):
    if x == x and x is not None:
        if x == 0:
            return 2
        if abs(x) <= 3:
            return 1
        return 0
    return 0.5
rdf['register_time_diff_sparse'] = rdf.register_time_diff.map(register_time_diff_sparse)

In [174]:
set(rdf.is_import1), set(rdf.is_import2)

({0.0, 1.0, nan, nan, nan, nan, nan, nan},
 {0.0, 1.0, nan, nan, nan, nan, nan, nan})

In [175]:
rdf['is_import1'] = rdf.is_import1.map(lambda x: x if x == x else -1)
rdf['is_import2'] = rdf.is_import2.map(lambda x: x if x == x else -1)

In [177]:
def is_import_diff_sparse(row):
    if row.is_import1 == -1 or row.is_import2 == -1:
        return 0.5
    if row.is_import1 == row.is_import2:
        return 2
    return 0
rdf['is_import_diff_sparse'] = rdf.apply(is_import_diff_sparse, axis=1)

In [178]:
bar_to_show('is_import_diff_sparse')

In [181]:
set(rdf.transfer_times1), set(rdf.transfer_times2)

({0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan, nan, nan, nan, nan, nan, 7.0},
 {0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, nan, nan, nan, nan, nan, nan, 7.0, 8.0})

In [182]:
rdf['transfer_times1'] = rdf.transfer_times1.map(lambda x: x if x == x else -1)
rdf['transfer_times2'] = rdf.transfer_times2.map(lambda x: x if x == x else -1)

In [183]:
set(rdf.transfer_times1), set(rdf.transfer_times2)

({-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0},
 {-1.0, 0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0})

In [186]:
def transfer_times_diff(row):
    if row.transfer_times1 == -1 or row.transfer_times2 == -1:
        return None
    return row.transfer_times1 - row.transfer_times2

rdf['transfer_times_diff'] = rdf.apply(transfer_times_diff, axis=1)

In [187]:
bar_to_show('transfer_times_diff')

In [188]:
def transfer_times_diff_sparse(x):
    if x == x or x is not None:
        if x== 0:
            return 2
        if abs(x) <2:
            return 1
        return 0
    return 0.5
rdf['transfer_times_diff_sparse'] = rdf.transfer_times_diff.map(transfer_times_diff_sparse)

In [189]:
bar2show = bar_to_show

In [190]:
bar2show('transfer_times_diff_sparse')

In [192]:
df.columns

Index(['id', 'color1', 'color2', 'displacement1', 'displacement2', 'mile1',
       'mile2', 'year1', 'year2', 'licensed_city1', 'licensed_city2', 'title1',
       'title2', 'fuel_type1', 'fuel_type2', 'register_time1',
       'register_time2', 'owner_price1', 'owner_price2', 'is_import1',
       'is_import2', 'transfer_times1', 'transfer_times2',
       'displacement_standard1', 'displacement_standard2', 'gearbox1',
       'gearbox2', 'is_same'],
      dtype='object')

In [193]:
set(rdf.displacement_standard1), set(rdf.displacement_standard2)

({nan, '国一', '国三', '国二', '国五', '国四'}, {nan, '国一', '国三', '国二', '国五', '国四'})

In [194]:
displacement_standard_map = {'国一': 1, '国三':3, '国二':2, '国五':5, '国四':4, '国六': 6}        
rdf['displacement_standard1'] = rdf.displacement_standard1.map(lambda x: displacement_standard_map.get(x, -1))
rdf['displacement_standard2'] = rdf.displacement_standard2.map(lambda x: displacement_standard_map.get(x, -1))

In [195]:
set(rdf.displacement_standard1), set(rdf.displacement_standard2)

({-1, 1, 2, 3, 4, 5}, {-1, 1, 2, 3, 4, 5})

In [196]:
set(rdf.gearbox1), set(rdf.gearbox2) 

({'AT', 'MT', nan}, {'AT', 'MT', nan, '手动', '自动'})

In [197]:
gearbox_map = {
    'AT': 'AT',
    'MT': 'MT',
    '手动': 'MT',
    '自动': 'AT'
}
rdf['gearbox1'] = rdf.gearbox1.map(lambda x: gearbox_map.get(x, -1))
rdf['gearbox2'] = rdf.gearbox2.map(lambda x: gearbox_map.get(x, -1))

In [198]:
set(rdf.gearbox1), set(rdf.gearbox2) 

({-1, 'AT', 'MT'}, {-1, 'AT', 'MT'})

In [199]:
rdf.to_csv('processed_same_car.csv', index=None)

In [201]:
list(rdf.columns)

['id',
 'color1',
 'color2',
 'displacement1',
 'displacement2',
 'mile1',
 'mile2',
 'year1',
 'year2',
 'licensed_city1',
 'licensed_city2',
 'title1',
 'title2',
 'fuel_type1',
 'fuel_type2',
 'register_time1',
 'register_time2',
 'owner_price1',
 'owner_price2',
 'is_import1',
 'is_import2',
 'transfer_times1',
 'transfer_times2',
 'displacement_standard1',
 'displacement_standard2',
 'gearbox1',
 'gearbox2',
 'is_same',
 'displacement_diff',
 'displacement_diff_sparse',
 'mile_diff',
 'mile_diff_rate',
 'mile_diff_rate_sparse',
 'mile_diff_sparse',
 'year_diff',
 'year_diff_sparse',
 'licensed_city_diff_sparse',
 'title_diff',
 'title_diff_sparse',
 'register_time_diff',
 'register_time_diff_sparse',
 'owner_price_diff',
 'is_import_diff_sparse',
 'transfer_times_diff',
 'transfer_times_diff_sparse']